In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV
!pip install xgboost
import string
import contractions
from nltk import word_tokenize          
from nltk.stem import WordNetLemmatizer

from xgboost import XGBClassifier

# Reading data files

In [2]:
data = pd.read_csv('TA_Training_Set.csv', error_bad_lines=False, engine='python') 
test_data= pd.read_csv('TA_Test_Set.csv', error_bad_lines=False, engine='python')

In [3]:
data=data.sample(n=100000)

In [4]:
data.head()

,Comment,Topic
53871,Here you go.\nTwitter: https://twitter.com/MDF...,2
565102,It's okay because they're both good looking.,30
24065,Earthmelon sounds like a really juicy mushroom...,18
798821,You are doing God's work,29
225268,Her ultra wealthy rich super genius husband?,30


In [5]:
test_data

,Comment
0,I bought a month and a half out on a stock tha...
1,"Parity used to be the justification, but that ..."
2,Yeah cartel. Legolas is gonna shoot your ass d...
3,"I do think he’s TA, but there’s one thing with..."
4,"Were trying, let you know if anything works"
...,...
99995,As I migraine sufferer I can tell you coffee i...
99996,If she was a drink she'd be room temperature t...
99997,I live on campus here. And was there when it h...
99998,bro nobody likes that shit


# Data Cleaning

In [6]:

data=data.loc[data['Comment'].str.contains(r'[^\x00-\x7F]+') == False]
data

,Comment,Topic
53871,Here you go.\nTwitter: https://twitter.com/MDF...,2
565102,It's okay because they're both good looking.,30
24065,Earthmelon sounds like a really juicy mushroom...,18
798821,You are doing God's work,29
225268,Her ultra wealthy rich super genius husband?,30
...,...,...
761751,"Ah, english is not my first language, my brows...",17
403292,You mean crying about other people being close...,3
856433,So my theory is that King Crimson is The Boss'...,2
783972,Obama is a biracial man who identifies as blac...,28


In [7]:
test_data

,Comment
0,I bought a month and a half out on a stock tha...
1,"Parity used to be the justification, but that ..."
2,Yeah cartel. Legolas is gonna shoot your ass d...
3,"I do think he’s TA, but there’s one thing with..."
4,"Were trying, let you know if anything works"
...,...
99995,As I migraine sufferer I can tell you coffee i...
99996,If she was a drink she'd be room temperature t...
99997,I live on campus here. And was there when it h...
99998,bro nobody likes that shit


In [8]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop = stopwords.words('english')

data['x_without_stopwords'] = data['Comment'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
test_data['x_without_stopwords'] = test_data['Comment'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\scherukuri1\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
test_data

,Comment,x_without_stopwords
0,I bought a month and a half out on a stock tha...,I bought month half stock almost option volume...
1,"Parity used to be the justification, but that ...","Parity used justification, days free agency. I..."
2,Yeah cartel. Legolas is gonna shoot your ass d...,Yeah cartel. Legolas gonna shoot ass
3,"I do think he’s TA, but there’s one thing with...","I think he’s TA, there’s one thing you’re sayi..."
4,"Were trying, let you know if anything works","Were trying, let know anything works"
...,...,...
99995,As I migraine sufferer I can tell you coffee i...,As I migraine sufferer I tell coffee catalyst ...
99996,If she was a drink she'd be room temperature t...,If drink she'd room temperature tap water
99997,I live on campus here. And was there when it h...,I live campus here. And happened. Super fucked
99998,bro nobody likes that shit,bro nobody likes shit


In [10]:
symbols = "!\"#$%&()*+-./:;<=>?@[\]^_`{|}~\nÄô"
for i in symbols:
    data["x_without_stopwords"] = data["x_without_stopwords"].str.replace(i,'')
    data["x_without_stopwords"] = data["x_without_stopwords"].str.replace('\d+', '')
    test_data["x_without_stopwords"] = test_data["x_without_stopwords"].str.replace(i,'')
    test_data["x_without_stopwords"] = test_data["x_without_stopwords"].str.replace('\d+', '')

<ipython-input-10-cee0bf171ffb>:4: FutureWarning: The default value of regex will change from True to False in a future version.
  data["x_without_stopwords"] = data["x_without_stopwords"].str.replace('\d+', '')
<ipython-input-10-cee0bf171ffb>:6: FutureWarning: The default value of regex will change from True to False in a future version.
  test_data["x_without_stopwords"] = test_data["x_without_stopwords"].str.replace('\d+', '')
<ipython-input-10-cee0bf171ffb>:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  data["x_without_stopwords"] = data["x_without_stopwords"].str.replace(i,'')
<ipython-input-10-cee0bf171ffb>:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  test_data["x_w

# Train Test Split

In [11]:
X_train, X_test, y_train, y_test = train_test_split(
    data['x_without_stopwords'], 
    data['Topic'], 
    random_state=11, 
    test_size=0.1)

In [12]:
X_train.shape



(74178,)

In [13]:
X_test.head(4)

618254    Idk, Brienne riding horse  pages AFfC pretty c...
195318                       We watch career great interest
417217    Yeah damn rockets fault though delivering nuke...
376174           Don't forget watch league gets like  views
Name: x_without_stopwords, dtype: object

In [14]:
test_data_prof = test_data["x_without_stopwords"]
test_data_prof.head(5)

0    I bought month half stock almost option volume...
1    Parity used justification, days free agency It...
2                  Yeah cartel Legolas gonna shoot ass
3    I think he’s TA, there’s one thing you’re sayi...
4                 Were trying, let know anything works
Name: x_without_stopwords, dtype: object

In [15]:
test_data.head(5)

,Comment,x_without_stopwords
0,I bought a month and a half out on a stock tha...,I bought month half stock almost option volume...
1,"Parity used to be the justification, but that ...","Parity used justification, days free agency It..."
2,Yeah cartel. Legolas is gonna shoot your ass d...,Yeah cartel Legolas gonna shoot ass
3,"I do think he’s TA, but there’s one thing with...","I think he’s TA, there’s one thing you’re sayi..."
4,"Were trying, let you know if anything works","Were trying, let know anything works"


# Lemmatizing

In [16]:
class LemmaTokenizer(object):
    def __init__(self):
        self.wnl = WordNetLemmatizer()
    def __call__(self, text):
        text = contractions.fix(text)
        return [self.wnl.lemmatize(t) for t in word_tokenize(text) if t not in string.punctuation]

# Modelling

In [17]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier
import nltk
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression

nltk.download('wordnet')
nltk.download('punkt')

xg_clf = Pipeline([
    ('vect', CountVectorizer(tokenizer=LemmaTokenizer(),stop_words='english')), # returns matrix of token counts
    ('tfidf', TfidfTransformer()), # transform a count matrix to a normalized tf or tf-idf representation
    ('clf', LogisticRegression( penalty='l2', tol=0.0001, C=1.0, fit_intercept=True, intercept_scaling=1,
                               class_weight=None, random_state=None, solver='lbfgs', max_iter=1000, multi_class='auto',
                               verbose=0, warm_start=False, n_jobs=None, l1_ratio=None)
    ),
])

xg_clf_rf = Pipeline([
    ('vect', CountVectorizer(tokenizer=LemmaTokenizer(),stop_words='english')), # returns matrix of token counts
    ('tfidf', TfidfTransformer()), # transform a count matrix to a normalized tf or tf-idf representation
    ('clf', RandomForestClassifier(n_estimators=400)
    ),
])

xg_clf_svm= Pipeline([
    ('vect', CountVectorizer(tokenizer=LemmaTokenizer(),stop_words='english')), # returns matrix of token counts
    ('tfidf', TfidfTransformer()), # transform a count matrix to a normalized tf or tf-idf representation
    ('clf', svm.SVC(kernel='sigmoid')
    ),
])

xg_clf_dt= Pipeline([
    ('vect', CountVectorizer(tokenizer=LemmaTokenizer(),stop_words='english')), # returns matrix of token counts
    ('tfidf', TfidfTransformer()), # transform a count matrix to a normalized tf or tf-idf representation
    ('clf', DecisionTreeClassifier()
    ),
])

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\scherukuri1\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\scherukuri1\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# LOGISTIC REGRESSION

In [18]:
xg_clf.fit(X_train, y_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


Pipeline(steps=[('vect',
                 CountVectorizer(stop_words='english',
                                 tokenizer=<__main__.LemmaTokenizer object at 0x00000256CD7C6CA0>)),
                ('tfidf', TfidfTransformer()),
                ('clf', LogisticRegression(max_iter=1000))])

In [19]:
predict_lr = xg_clf.predict(X_test)

In [20]:
predict_lr_test_data = xg_clf.predict(test_data_prof)

In [21]:
predict_lr_test_data_df=pd.DataFrame(predict_lr_test_data)

In [22]:
test_data

,Comment,x_without_stopwords
0,I bought a month and a half out on a stock tha...,I bought month half stock almost option volume...
1,"Parity used to be the justification, but that ...","Parity used justification, days free agency It..."
2,Yeah cartel. Legolas is gonna shoot your ass d...,Yeah cartel Legolas gonna shoot ass
3,"I do think he’s TA, but there’s one thing with...","I think he’s TA, there’s one thing you’re sayi..."
4,"Were trying, let you know if anything works","Were trying, let know anything works"
...,...,...
99995,As I migraine sufferer I can tell you coffee i...,As I migraine sufferer I tell coffee catalyst ...
99996,If she was a drink she'd be room temperature t...,If drink she'd room temperature tap water
99997,I live on campus here. And was there when it h...,I live campus here And happened Super fucked
99998,bro nobody likes that shit,bro nobody likes shit


In [23]:
my_new_df = pd.concat([test_data, predict_lr_test_data_df], axis =1)

my_new_df.to_csv("Topic_Analysis_Output.csv")

In [24]:
# Accuracy
print('Accuracy =', np.mean(predict_lr == y_test))

Accuracy = 0.4470046705002565


In [25]:
print(confusion_matrix(y_test, predict_lr))
print(classification_report(
    y_test, 
    predict_lr))

[[1037   11   32 ...   28   15   14]
 [  11  891   33 ...   44   31   13]
 [  26   54  334 ...   58   36   27]
 ...
 [  15   42   42 ...  639   42   39]
 [  11   42   28 ...   18 1238   13]
 [  16   21   31 ...   44   27  659]]
              precision    recall  f1-score   support

           1       0.65      0.60      0.62      1728
           2       0.35      0.43      0.39      2052
           3       0.21      0.18      0.19      1888
           4       0.36      0.36      0.36      1913
           5       0.50      0.48      0.49      1797
           6       0.69      0.83      0.75      1705
           7       0.61      0.61      0.61      1803
           8       0.23      0.11      0.15        27
           9       0.40      0.50      0.44      1925
          10       0.37      0.38      0.38      1929
          11       0.70      0.63      0.66      1969
          12       0.35      0.30      0.32      1929
          13       0.57      0.57      0.57      1860
          14   

In [26]:
print("The training accuracy is :")
print(classification_report(
    y_train, 
    xg_clf.predict(X_train)))

The training accuracy is :
              precision    recall  f1-score   support

           1       0.71      0.66      0.68     15680
           2       0.44      0.54      0.49     17987
           3       0.35      0.29      0.32     16698
           4       0.48      0.48      0.48     17684
           5       0.59      0.55      0.57     16556
           6       0.71      0.85      0.78     15155
           7       0.67      0.66      0.66     16614
           8       0.51      0.14      0.21       361
           9       0.47      0.60      0.53     17144
          10       0.47      0.46      0.47     17330
          11       0.74      0.68      0.70     17287
          12       0.50      0.45      0.47     17680
          13       0.63      0.63      0.63     16801
          14       0.56      0.59      0.57     17201
          15       0.50      0.57      0.54     17061
          16       0.64      0.49      0.55     18313
          17       0.51      0.43      0.47     16806


# RANDOM FOREST

In [23]:
xg_clf_rf.fit(X_train, y_train)

predict_rf = xg_clf_rf.predict(X_test)

# Accuracy
print('Accuracy =', np.mean(predict_rf == y_test))

print(confusion_matrix(y_test, predict_rf))
print(classification_report(
    y_test, 
    predict_rf))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


Accuracy = 0.3214848962756278
[[103   3   7 ...   4   2   4]
 [  1  64   3 ...   2   2   0]
 [  1   9  24 ...   3   8   6]
 ...
 [  1   6   1 ...  40   1   7]
 [  1   8   4 ...   4  96   2]
 [  1   0   1 ...   3   1  44]]
              precision    recall  f1-score   support

           1       0.66      0.50      0.57       204
           2       0.27      0.27      0.27       233
           3       0.16      0.11      0.13       226
           4       0.18      0.15      0.16       228
           5       0.33      0.34      0.34       193
           6       0.43      0.74      0.55       165
           7       0.54      0.52      0.53       229
           8       0.00      0.00      0.00         1
           9       0.31      0.39      0.34       220
          10       0.35      0.30      0.32       204
          11       0.50      0.47      0.48       205
          12       0.23      0.15      0.18       201
          13       0.38      0.40      0.39       220
          14       0.

# SVM

In [18]:
xg_clf_svm.fit(X_train, y_train)

predict_svm = xg_clf_svm.predict(X_test)

# Accuracy
print('Accuracy =', np.mean(predict_svm == y_test))

print(confusion_matrix(y_test, predict_svm))
print(classification_report(
    y_test, 
    predict_svm))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


Accuracy = 0.34720368797767803
[[105   0   5 ...   1   2   2]
 [  0  64  11 ...   5   1   4]
 [  2  12  53 ...   4   4   2]
 ...
 [  1   6   8 ...  55   2   5]
 [  4  11  10 ...   5  90   2]
 [  4   1   8 ...   8   1  55]]
              precision    recall  f1-score   support

           1       0.64      0.51      0.57       204
           2       0.25      0.27      0.26       233
           3       0.16      0.23      0.19       226
           4       0.23      0.22      0.23       228
           5       0.35      0.36      0.35       193
           6       0.66      0.75      0.70       165
           7       0.59      0.53      0.56       229
           8       0.00      0.00      0.00         1
           9       0.36      0.41      0.38       220
          10       0.33      0.36      0.35       204
          11       0.59      0.45      0.51       205
          12       0.23      0.21      0.22       201
          13       0.45      0.46      0.46       220
          14       0

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# DECISION TREE

In [19]:
xg_clf_dt.fit(X_train, y_train)

predict_dt = xg_clf_dt.predict(X_test)

# Accuracy
print('Accuracy =', np.mean(predict_dt == y_test))

print(confusion_matrix(y_test, predict_dt))
print(classification_report(
    y_test, 
    predict_dt))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


Accuracy = 0.23923328885114642
[[94  3  4 ...  2  4  4]
 [ 1 53  2 ...  5  3  1]
 [ 4 10 18 ...  0  5  4]
 ...
 [ 3  7  4 ... 38  1  7]
 [ 2  5  3 ...  4 74  4]
 [ 1  1  6 ...  6  2 31]]
              precision    recall  f1-score   support

           1       0.50      0.46      0.48       204
           2       0.24      0.23      0.23       233
           3       0.09      0.08      0.09       226
           4       0.13      0.11      0.12       228
           5       0.26      0.28      0.27       193
           6       0.47      0.58      0.51       165
           7       0.42      0.37      0.39       229
           8       0.00      0.00      0.00         1
           9       0.24      0.26      0.25       220
          10       0.27      0.24      0.26       204
          11       0.33      0.37      0.35       205
          12       0.14      0.13      0.14       201
          13       0.29      0.24      0.26       220
          14       0.26      0.26      0.26       211
  

# MLPClassifier

In [20]:
from sklearn.neural_network import MLPClassifier

text_tfidf = Pipeline([
    ('vect', CountVectorizer(tokenizer=LemmaTokenizer(),stop_words='english')), 
    ('tfidf', TfidfTransformer()), 
])

text_tfidf.fit(X_train)
X_train_tfidf = text_tfidf.transform(X_train)
X_test_tfidf = text_tfidf.transform(X_test)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


In [21]:
MLP_clf = MLPClassifier(
    max_iter=800, 
    activation='logistic',
    solver='lbfgs',
    random_state=10)

MLP_clf.fit(X_train_tfidf, y_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:500: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


MLPClassifier(activation='logistic', max_iter=800, random_state=10,
              solver='lbfgs')

In [22]:
predict_MLP = MLP_clf.predict(X_test_tfidf)
print(confusion_matrix(y_test, predict_MLP))
# Accuracy
print('Accuracy =', np.mean(predict_MLP == y_test))

[[101   0   8 ...   3   1   2]
 [  1  59  10 ...   4   2   7]
 [  6  11  32 ...  10   3   5]
 ...
 [  3   1   6 ...  50   2   6]
 [  4   7   9 ...   3  81   2]
 [  3   2  10 ...   7   3  41]]
Accuracy = 0.2980710906223462
